# Fooocus: код с GitHub + модели с Google Drive

**Схема:** репозиторий на GitHub (только код), модели лежат на Drive. В Colab клонируем код и подключаем папку с моделями с Drive.

## Перед запуском

1. Выгрузите **код** Fooocus на GitHub (без папок `models`, `venv`, см. `.gitignore`).
2. Один раз залейте папку **моделей** (`models 12.07.29` или `models`) в Google Drive.
3. **Runtime → Change runtime type → GPU** (T4).
4. Ниже укажите: ссылку на ваш репозиторий и путь к папке с моделями на Drive.
5. Запустите ячейки по порядку.

In [ ]:
# Ваш репозиторий Fooocus на GitHub (HTTPS или SSH)
GITHUB_REPO = "https://github.com/lolikoni/fooocus.git"

# Путь на Google Drive к папке, где лежат модели:
# либо папка Fooocus целиком (внутри есть "models 12.07.29" или "models"),
# либо папка, внутри которой сразу лежит "models 12.07.29" или "models"
# Примеры: "Fooocus"  или  "AI-modeli/Fooocus"  или  "Fooocus_models"
DRIVE_PATH = "Fooocus"

In [ ]:
# Клонируем код с GitHub
import os
import shutil

REPO_DIR = "/content/Fooocus"
if os.path.isdir(REPO_DIR):
    shutil.rmtree(REPO_DIR)

get_ipython().system(f'git clone --depth 1 {GITHUB_REPO} {REPO_DIR}')

FOOCUS_ROOT = REPO_DIR
print(f"Код склонирован: {FOOCUS_ROOT}")

In [ ]:
# Подключаем Drive и подтягиваем папку с моделями (симлинк — без копирования)
from google.colab import drive

drive.mount("/content/drive", use_metadata_cache=False)

drive_base = os.path.join("/content/drive/MyDrive", DRIVE_PATH.strip("/"))
if not os.path.isdir(drive_base):
    raise FileNotFoundError(f"На Drive не найдена папка: {drive_base}")

# Ищем папку с чекпоинтами: "models 12.07.29" или "models"
models_12 = os.path.join(drive_base, "models 12.07.29")
models_std = os.path.join(drive_base, "models")
if os.path.isdir(models_12):
    drive_models_dir = models_12
    models_folder_name = "models 12.07.29"
elif os.path.isdir(models_std):
    drive_models_dir = models_std
    models_folder_name = "models"
else:
    raise FileNotFoundError(
        f"В папке {drive_base} не найдены ни \"models 12.07.29\", ни \"models\". "
        "Загрузите на Drive папку с чекпоинтами."
    )

target_models = os.path.join(FOOCUS_ROOT, models_folder_name)
if os.path.exists(target_models):
    os.remove(target_models) if os.path.islink(target_models) else shutil.rmtree(target_models)
os.symlink(drive_models_dir, target_models)
print(f"Модели подключены: {target_models} -> {drive_models_dir}")

path_checkpoints = os.path.join(target_models, "checkpoints")
if os.path.isdir(path_checkpoints):
    models = [f for f in os.listdir(path_checkpoints) if f.endswith((".safetensors", ".ckpt"))]
    print(f"Чекпоинтов: {len(models)}")
    for m in models[:8]:
        print(f"  - {m}")
    if len(models) > 8:
        print(f"  ... и ещё {len(models) - 8}")
else:
    print("Внимание: папка checkpoints не найдена внутри моделей на Drive.")

In [ ]:
# Прописываем пути в config.txt для Colab
import json

models_base = models_folder_name
models_dir = os.path.join(FOOCUS_ROOT, models_base)
config_path = os.path.join(FOOCUS_ROOT, "config.txt")

paths = {
    "path_checkpoints": [os.path.join(models_dir, "checkpoints")],
    "path_loras": [os.path.join(models_dir, "loras")],
    "path_embeddings": os.path.join(models_dir, "embeddings"),
    "path_vae_approx": os.path.join(models_dir, "vae_approx"),
    "path_vae": os.path.join(models_dir, "vae"),
    "path_outputs": os.path.join(FOOCUS_ROOT, "outputs"),
}
for key, val in paths.items():
    if isinstance(val, list):
        for v in val:
            os.makedirs(v, exist_ok=True)
    else:
        os.makedirs(val, exist_ok=True)

config = {} if not os.path.isfile(config_path) else json.load(open(config_path, "r", encoding="utf-8"))
config.update(paths)
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4, ensure_ascii=False)
print("config.txt обновлён.")

In [ ]:
# Запуск Fooocus
os.chdir(FOOCUS_ROOT)

!pip install -q pygit2==1.15.1
!python launch.py --share --always-high-vram